
##**ChipNet**

This notebook demonstrates the implementation of the paper - **ChipNet: Budget-Aware Pruning with Heaviside Continuous Approximations**

Steps to train a baseline model and then compress it for a given budget are as follows: 
*   Load the YAML file.
*   Load dataset and create dataloaders.
*   Create ChipNet object and pass the parameters in the form of a dictionary.
*   Pass the dataloaders into the compress_model method to obtain the compressed model.

Since this is a demo notebook the number of epochs have been set to 1, 1 and 2 respectively for pretraining, pruning and finetuning respectively.



In [22]:
import torch
import matplotlib.pyplot as plt
from torchvision import transforms
from trailmet.models import ModelsFactory
from trailmet.datasets.classification import DatasetFactory
import yaml

In [23]:
root = "/content/trailmet/experiments/pruning/chipnet"

###Loading the yaml file for the configurations.

In [24]:
import os
with open(os.path.join(root, "chipnet.yaml"), 'r') as stream:
    data_loaded = yaml.safe_load(stream)
print(data_loaded)

{'CHIPNET_ARGS': {'BUDGET_TYPE': 'channel_ratio', 'TARGET_BUDGET': 0.5, 'BUDGET_LOSS_WEIGHTAGE': 30, 'CRISPNESS_LOSS_WEIGHTAGE': 10, 'BETA_INCREMENT': 5, 'GAMMA_INCREMENT': 2, 'STEEPNESS': 100}, 'PRETRAIN': {'EPOCHS': 1, 'TEST_ONLY': False, 'OPTIMIZER': 'SGD', 'LR': 0.05, 'SCHEDULER_TYPE': 1, 'WEIGHT_DECAY': 0.001}, 'PRUNE': {'EPOCHS': 1, 'TEST_ONLY': False, 'OPTIMIZER': 'SGD', 'LR': 0.05, 'WEIGHT_DECAY': 0.001}, 'FINETUNE': {'EPOCHS': 2, 'TEST_ONLY': False, 'OPTIMIZER': 'SGD', 'LR': 0.05, 'SCHEDULER_TYPE': 1, 'WEIGHT_DECAY': 0.001}}


###Loading the model.

In [25]:
model = ModelsFactory.create_model('resnet50', 100, False, insize=32)

###Loading the dataset.

In [26]:
from trailmet.datasets.classification import DatasetFactory
data_root = "/content/data_dir"

In [27]:
mkdir /content/data_dir

mkdir: cannot create directory ‘/content/data_dir’: File exists


In [28]:
train_transform = transforms.Compose(
[transforms.ToTensor()])

val_transform = transforms.Compose(
[transforms.ToTensor()])

test_transform = transforms.Compose(
[transforms.ToTensor()])

transforms1 = {
    'train': train_transform, 
    'val': val_transform, 
    'test': test_transform}
def train_target_transform(label):
    return label

def val_target_transform(label):
    return label

def test_target_transform(label):
    return label

target_transforms = {
    'train': None, 
    'val': None, 
    'test': None}


cifar_dataset = DatasetFactory.create_dataset(name = 'CIFAR100', 
                                        root = data_root,
                                        split_types = ['train', 'val', 'test'],
                                        val_fraction = 0.2,
                                        transform = transforms1,
                                        target_transform = target_transforms
)


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


###Creating the dataloaders.

In [29]:
train_loader = torch.utils.data.DataLoader(
        cifar_dataset['train'], batch_size=64, 
        sampler=cifar_dataset['train_sampler'],
        num_workers=0
    )
val_loader = torch.utils.data.DataLoader(
        cifar_dataset['val'], batch_size=64, 
        sampler=cifar_dataset['val_sampler'],
        num_workers=0
    )
test_loader = torch.utils.data.DataLoader(
        cifar_dataset['test'], batch_size=64, 
        sampler=cifar_dataset['test_sampler'],
        num_workers=0
    )

###Creating the method's object followed by compression

In [30]:
from trailmet.algorithms.prune.chipnet import ChipNet

In [31]:
a = ChipNet(model, {'train': train_loader, 'val': val_loader, 'test': test_loader}, **data_loaded)

In [32]:
a.compress_model()

100%|██████████| 157/157 [00:05<00:00, 31.18it/s, acc1=10.2, acc5=32.7, loss=3.83]


**Saving model**


100%|██████████| 157/157 [00:05<00:00, 28.25it/s, acc1=10.3, acc5=32.1, loss=3.85]


Test Accuracy: 10.300557324840764 | Valid Accuracy: 10.161226114649681
preparing model for pruning
Starting epoch 1 / 1


100%|██████████| 625/625 [02:35<00:00,  4.02it/s, loss=5.63]


...

**Saving model**


100%|██████████| 157/157 [00:06<00:00, 23.92it/s, acc1=4.04, acc5=16.6, loss=4.26]


**Saving model**


100%|██████████| 157/157 [00:06<00:00, 22.58it/s, acc1=4.1, acc5=16.4, loss=4.27]

Test Accuracy: 4.100318471337579 | Valid Accuracy: 4.040605095541402
